In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
import tensorflow as tf
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np

In [11]:
def apply_region_filter(before, after):
    before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
    after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)
    (score, diff) = ssim(before_gray, after_gray, full=True)
    diff = (diff * 255).astype("uint8")
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    before_region = np.zeros(before.shape, np.uint8)
    after_region = np.zeros(after.shape, np.uint8)

    for c in contours:
        area = cv2.contourArea(c)
        if area > 40:
            x, y, w, h = cv2.boundingRect(c)
            before_region[y:y + h, x:x + w] = before[y:y + h, x:x + w]
            after_region[y:y + h, x:x + w] = after[y:y + h, x:x + w]

    return before_region, after_region

In [12]:
def create_conv_layers(input_img, input_shape):
    model = layers.Conv2D(32, (3, 3), padding='same', input_shape=input_shape)(input_img)
    model = layers.LeakyReLU(alpha=0.1)(model)
    model = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(model)
    model = layers.Dropout(0.25)(model)

    model = layers.Conv2D(64, (3, 3), padding='same')(model)
    model = layers.LeakyReLU(alpha=0.1)(model)
    model = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(model)
    model = layers.Dropout(0.25)(model)

    model = layers.Conv2D(96, (3, 3), padding='same')(model)
    model = layers.LeakyReLU(alpha=0.1)(model)
    model = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(model)
    model = layers.Dropout(0.25)(model)

    model = layers.Conv2D(128, (3, 3), padding='same')(model)
    model = layers.LeakyReLU(alpha=0.1)(model)
    model = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(model)
    model = layers.Dropout(0.25)(model)

    model = layers.Conv2D(256, (3, 3), padding='same')(model)
    model = layers.LeakyReLU(alpha=0.1)(model)
    model = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(model)
    model = layers.Dropout(0.4)(model)

    return model

In [13]:
as_gray = False  # use rgb or gray images?
in_channels = 3  # gray = 1, rgb = 4
if as_gray:
    in_channels = 1

img_height, img_width = 584, 480  # original

num_classes = 2  # click or no click
batch_size = 8
epochs = 2

train_dir_1 = "dataset/train/before"
train_dir_2 = "dataset/train/after"

test_dir_1 = "dataset/test/before"
test_dir_2 = "dataset/test/after"

In [14]:
def generate_generator_multiple(generator, dir1, dir2, batch_size, img_height, img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size=(img_height, img_width),
                                          class_mode='categorical',
                                          batch_size=batch_size,
                                          shuffle=True,
                                          seed=666)

    genX2 = generator.flow_from_directory(dir2,
                                          target_size=(img_height, img_width),
                                          class_mode='categorical',
                                          batch_size=batch_size,
                                          shuffle=True,
                                          seed=666)

    while True:
        before_imgs = []
        after_imgs = []

        X1i = genX1.next()
        X2i = genX2.next()

        for idx in range(len(X1i[0])):
            before = np.reshape(X1i[0][idx], (584, 480, 3))
            after = np.reshape(X2i[0][idx], (584, 480, 3))
            before, after = apply_region_filter(before.copy(), after.copy())

            before = np.asarray(before)
            before = before.astype('float32')
            before /= 255.0

            after = np.asarray(before)
            after = after.astype('float32')
            after /= 255.0

            before_imgs.append(before)
            after_imgs.append(after)

        before = np.reshape(before_imgs, (len(X1i[0]), 584, 480, 3))
        after = np.reshape(after_imgs, (len(X1i[0]), 584, 480, 3))
        yield [before, after], X2i[1]

In [15]:
input_shape = (img_height, img_width, in_channels)

before_input = layers.Input(shape=input_shape)
before_model = create_conv_layers(before_input, input_shape)

after_input = layers.Input(shape=input_shape)
after_model = create_conv_layers(after_input, input_shape)

conv = layers.concatenate([before_model, after_model])
conv = layers.Flatten()(conv)

dense = layers.Dense(512)(conv)
dense = layers.LeakyReLU(alpha=0.1)(dense)
dense = layers.Dropout(0.25)(dense)

dense = layers.Dense(256)(dense)
dense = layers.LeakyReLU(alpha=0.1)(dense)
dense = layers.Dropout(0.25)(dense)

dense = layers.Dense(128)(dense)
dense = layers.LeakyReLU(alpha=0.1)(dense)
dense = layers.Dropout(0.4)(dense)

output = layers.Dense(num_classes, activation='softmax')(dense)

model = Model(inputs=[before_input, after_input], outputs=[output])

opt = optimizers.Adam()

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['categorical_accuracy', 'accuracy'])

model_checkpoint = tf.keras.callbacks.ModelCheckpoint('ClickDetector{epoch:02d}.h5', period=1, save_weights_only=True)
callbacks = [model_checkpoint]

generator = ImageDataGenerator(rescale=1,  # 1./255,
                               rotation_range=0,
                               width_shift_range=0,
                               height_shift_range=0,
                               shear_range=0,
                               zoom_range=0,
                               horizontal_flip=False,
                               vertical_flip=False
                               )

inputgenerator = generate_generator_multiple(generator=generator,
                                             dir1=train_dir_1,
                                             dir2=train_dir_2,
                                             batch_size=batch_size,
                                             img_height=img_height,
                                             img_width=img_width)

testgenerator = generate_generator_multiple(generator,
                                            dir1=test_dir_1,
                                            dir2=test_dir_2,
                                            batch_size=batch_size,
                                            img_height=img_height,
                                            img_width=img_width)

In [ ]:
history = model.fit_generator(inputgenerator,
                              steps_per_epoch=404 / batch_size,
                              epochs=epochs,
                              validation_data=testgenerator,
                              validation_steps=102 / batch_size,
                              # use_multiprocessing=True,
                              shuffle=False,
                              verbose=2,
                              callbacks=callbacks)

Epoch 1/2
51/50 - 32s - loss: 2.6879 - categorical_accuracy: 0.8272 - accuracy: 0.8272 - val_loss: 1.0008 - val_categorical_accuracy: 0.9412 - val_accuracy: 0.9412
Epoch 2/2
